In [1]:
import sys
sys.path.append('./Scripts')
import os 
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau 
from tensorflow.keras.callbacks import TerminateOnNaN
import pandas as pd
from model import ModelByTensorflow
from structure import ElementwiseDNN

In [2]:
example_train_result_dir = './Results/Ex_train_result/'
CUDA_VISIBLE_DEVICES = '-1'
EPOCHS = 2
EARLY_STOP = 20
REDUCE_LR_PATIENCE = 5
BATCH_SIZE_PER_REPLICA = 20000 
MODEL_CHECK_POINT_MONITOR = 'PCC_RMSE'

In [3]:
model_params = dict(
    label_colnames=['pKa_energy'],
    separate_radial_and_angular_feature=True, 
    input_size_per_element_radial=207,
    input_size_per_element_angular=1080,
    use_only_radial_feature=False,
    n_nodes=[500]*6, 
    output_size_per_element=10, 
    output_layer_style='fc', 
    output_layer_n_nodes=[256, 256, 256],
    lr=1e-3, 
    use_residual_dense=True,
    n_layers_per_res_dense=1, 
    use_spectral_norm=False, 
    l2_norm=0.01, 
    dropout=0.15, 
    dropout_input=0.05,
    target_elements=['H', 'C', 'N', 'O', 'P', 'S', 'Cl', 'Zn', 'DU'],
    n_radial_Rs=23, 
    n_angular_Rs=3, 
    n_thetas=8,
)

# make callbacks 
model_check_point = ModelCheckpoint(
    filepath=os.path.join(
        example_train_result_dir, 
        "model-epoch{epoch:03d}-" + MODEL_CHECK_POINT_MONITOR\
        + "{val_" + MODEL_CHECK_POINT_MONITOR + ":.2f}.h5"),
    monitor='val_' + MODEL_CHECK_POINT_MONITOR, 
    save_best_only=False, 
    save_weights_only=False, 
    save_freq='epoch',
    verbose=1
)

terminate_on_nan =TerminateOnNaN()

reduce_lr = ReduceLROnPlateau(
    monitor='val_' + MODEL_CHECK_POINT_MONITOR,
    factor=0.2,
    patience=REDUCE_LR_PATIENCE,
    verbose=1,
    mode='auto',
)

fit_params =  {
    'epochs': EPOCHS, 
    'earlystop': EARLY_STOP,
    'batch_size_per_replica': BATCH_SIZE_PER_REPLICA,
    'use_mirrored_strategy': False,
    'cuda_visible_device': CUDA_VISIBLE_DEVICES,
    'callbacks': [model_check_point, terminate_on_nan, reduce_lr]
}

In [4]:
dataset_valid_paths = ['./Features/feature_validset.tfrecords']
dataset_train_paths = ['./Features/feature_trainset.tfrecords']
dataset_valid = tf.data.TFRecordDataset(dataset_valid_paths)
dataset_train = tf.data.TFRecordDataset(dataset_train_paths)


model = ModelByTensorflow(network_cls=ElementwiseDNN, **model_params)

In [5]:
model.fit(X=dataset_train, validation_data=dataset_valid, **fit_params)


cuda_visible_device: -1
os.environ['CUDA_VISIBLE_DEVICES']: -1
gpu_devices: [], NUM_GPUS: 0
use_mirrored_strategy: False
global_batch_size:  20000
Epoch 1/2
      1/Unknown - 0s 101us/step - loss: 547.7557 - mse: 93.7906 - RMSE: 6.8593 - PCC: 0.0266 - PCC_RMSE: 2.7392
Epoch 00001: saving model to ./Results/Ex_train_result/model-epoch001-PCC_RMSE3.66.h5
1/1 [==============================] - 56s 56s/step - loss: 547.7557 - mse: 93.7906 - RMSE: 6.8593 - PCC: 0.0266 - PCC_RMSE: 2.7392 - val_loss: 548.6578 - val_mse: 99.1965 - val_RMSE: 9.3000 - val_PCC: -0.2473 - val_PCC_RMSE: 3.6631
Epoch 2/2
1/1 [==============================] - ETA: 0s - loss: 547.7465 - mse: 94.6344 - RMSE: 6.8770 - PCC: 0.0161 - PCC_RMSE: 2.7518
Epoch 00002: saving model to ./Results/Ex_train_result/model-epoch002-PCC_RMSE2.84.h5
1/1 [==============================] - 18s 18s/step - loss: 547.7465 - mse: 94.6344 - RMSE: 6.8770 - PCC: 0.0161 - PCC_RMSE: 2.7518 - val_loss: 547.7897 - val_mse: 51.2505 - val_RMSE: 6.179

In [6]:
model_path = os.path.join(example_train_result_dir, 'example_model.h5')
training_log_path = os.path.join(example_train_result_dir, 'log.csv')
model.save_model(model_path)
model.log.to_csv(training_log_path)